# Data Management with OpenACC

This version of the lab is intended for C/C++ programmers. The Fortran version of this lab is available [here](../Fortran/README.ipynb).

You will receive a warning five minutes before the lab instance shuts down. Remember to save your work! If you are about to run out of time, please see the [Post-Lab](#Post-Lab-Summary) section for saving this lab to view offline later.

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

---
Let's execute the cell below to display information about the GPUs running on the server. To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above.  If all goes well, you should see some output returned below the grey cell.

In [1]:
!nvidia-smi

Thu Jun 19 07:43:45 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   47C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

---

## Introduction

Our goal for this lab is to use the OpenACC Data Directives to properly manage our data.
  
  
  
![development_cycle.png](../images/development_cycle.png)

This is the OpenACC 3-Step development cycle.

**Analyze** your code, and predict where potential parallelism can be uncovered. Use profiler to help understand what is happening in the code, and where parallelism may exist.

**Parallelize** your code, starting with the most time consuming parts. Focus on maintaining correct results from your program.

**Optimize** your code, focusing on maximizing performance. Performance may not increase all-at-once during early parallelization.

We are currently tackling the **parallelize** step. We will include the OpenACC data directive to properly manage data within our parallelized code.

---

## Run the Code (With Managed Memory)

In the previous lab, we ran our code with CUDA Managed Memory, and achieved a considerable performance boost. However, managed memory is not compatible with all GPUs, and it performs it may performs worsemmer defined, proper memory management. Run the following script, and note the time the program takes to run. We are expecting that our own implementation which we will develop later in this lab will run a little better.

In [3]:
!nvc -fast -ta=tesla:managed -Minfo=accel -o laplace_managed jacobi.c laplace2d.c && ./laplace_managed

jacobi.c:
laplace2d.c:
calcNext:
     36, Generating copy(A[:n*m]) [if not already present]
         Generating Tesla code
         38, #pragma acc loop gang /* blockIdx.x */
             Generating implicit reduction(max:error)
         41, #pragma acc loop vector(128) /* threadIdx.x */
     36, Generating implicit copy(error) [if not already present]
         Generating copy(Anew[:n*m]) [if not already present]
     41, Loop is parallelizable
swap:
     52, Generating copy(A[:n*m],Anew[:n*m]) [if not already present]
         Generating Tesla code
         54, #pragma acc loop gang /* blockIdx.x */
         57, #pragma acc loop vector(128) /* threadIdx.x */
     57, Loop is parallelizable
Jacobi relaxation Calculation: 4096 x 4096 mesh
    0, 0.250000
  100, 0.002397
  200, 0.001204
  300, 0.000804
  400, 0.000603
  500, 0.000483
  600, 0.000403
  700, 0.000345
  800, 0.000302
  900, 0.000269
 total: 1.161982 s


### Optional: Analyze the Code

If you would like a refresher on the code files that we are working on, you may view both of them using the two links below.

[jacobi.c](jacobi.c)  
[laplace2d.c](laplace2d.c)  

### Optional: Profile the Code

In [4]:
!nsys profile -t openacc --stats=true --force-overwrite true -o laplace_managed ./laplace_managed

Jacobi relaxation Calculation: 4096 x 4096 mesh
    0, 0.250000
  100, 0.002397
  200, 0.001204
  300, 0.000804
  400, 0.000603
  500, 0.000483
  600, 0.000403
  700, 0.000345
  800, 0.000302
  900, 0.000269
 total: 1.254324 s
Processing events...
Saving temporary "/tmp/nsys-report-9e41-bff6-e732-d1bd.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-9e41-bff6-e732-d1bd.qdrep"
Exporting 71111 events: [=================================================100%]

Exported successfully to
/tmp/nsys-report-9e41-bff6-e732-d1bd.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls   Average    Minimum   Maximum           Name        
 -------  ---------------  ---------  ----------  --------  --------  --------------------
    90.5       1094611316       4000    273652.8         0  65812179  cuStreamSynchronize 
     2.8         33798463       3000     11266.2

Let's check out the profiler's report. Once the profiling run has completed, download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](laplace_managed.qdrep) (choose *save link as*), and open it via the GUI. To view the profiler report locally, please see the section on [How to view the report](../../../module2/English/C/README.ipynb#viewreport).

---

## OpenACC Structured Data Directive

The OpenACC data directives allow the programmer to explicitly manage the data on the device (in our case, the GPU). Specifically, the structured data directive will mark a static region of our code as a **data region**.

```cpp
< Initialize data on host (CPU) >

#pragma acc data < data clauses >
{

    < Code >

}
```

Device memory allocation happens at the beginning of the region, and device memory deallocation happens at the end of the region. Additionally, any data movement from the host to the device (CPU to GPU) happens at the beginning of the region, and any data movement from the device to the host (GPU to CPU) happens at the end of the region. Memory allocation/deallocation and data movement is defined by which clauses the programmer includes. This is a list of the most important data clauses that we can use:

**copy** : `copy( A[0:N] )` : Allocates memory on device and copies data from host to device when entering region and copies data back to the host when exiting region  
**copyin** : `copyin( A[0:N] )` : Allocates memory on device and copies data from host to device when entering region  
**copyout** : `copyout( A[0:N] )` : Allocates memory on device and copies data to the host when exiting region  
**create** : `create( A[0:N] )` : Allocates memory on device but does not copy  
**present** : `present( A )` : Data is already present on device from another containing data region  

All of these data clauses (except for present) will allocate device memory at the beginning of the data region, and deallocate device memory at the end of the data region. And with the exception of create, they will also transfer some amount of data between the host and device.

You may also use them to operate on multiple arrays at once, by including those arrays as a comma separated list.

```cpp
#pragma acc data copy( A[0:N], B[0:M], C[0:Q] )
```

You may also use more than one data clause at a time.

```cpp
#pragma acc data create( A[0:N] ) copyin( B[0:M] ) copyout( C[0:Q] )
```

These clauses can also be used directly with a parallel or kernels directive, because every parallel and kernels directive is surrounded by an **implied data region**.

```cpp
#pragma acc kernels create(A[0:N]) copyin(B[0:M]) present(C[0:Q])
{
    < Code that uses A, B, and C >
}
```

### Encompassing Multiple Compute Regions

A single data region can contain any number of parallel/kernels regions. Take the following example:

```cpp
#pragma acc data copyin(A[0:N], B[0:N]) create(C[0:N])
{

    #pragma acc parallel loop
    for( int i = 0; i < N; i++ )
    {
        C[i] = A[i] + B[i];
    }
    
    #pragma acc parallel loop
    for( int i = 0; i < N; i++ )
    {
        A[i] = C[i] + B[i];
    }

}
```

You may also encompass function calls within the data region:

```cpp
void copy(int *A, int *B, int N)
{
    #pragma acc parallel loop copyout(A[0:N]) copyin(B[0:N])
    for( int i = 0; i < N; i++ )
    {
        A[i] = B[i];
    }
}

...

#pragma acc data copyout(A[0:N],B[0:N]) copyin(C[0:N])
{
    copy(A, C, N);
    
    copy(A, B, N);
}
```

But wouldn't this code now result in my arrays being copied twice, once by the data region and then again by the parallel loop inside the function calls? In fact, the OpenACC runtime is smart enough to handle exactly this case. Data will be copied in only the first time its encountered in a data clause and out only the last time its encountered in a data clause. This allows you to create fully-working directives within your functions and then later *"hoist"* the data movement to a higher level without changing your code at all. This is part of incrementally accelerating your code to avoid incorrect results.

### Array Shaping

The *array shape* defines a portion of an array. Take the following example:

```cpp
int *A = (int*) malloc(N * sizeof(int));

#pragma acc data create( A[0:N] )
```

The array shape is defined as [0:N], this means that the GPU copy will start at index 0, and be of size N. Array shape is of the format **Array[starting_index:size]**. Let's look at an example where we only want a portion of the array.

```cpp
int *A = (int*) malloc(N * sizeof(int));

#pragma acc data create( A[0:N/2] )
```

In this example, the GPU copy will start at index 0, but will only be half the size of the CPU copy.

The shape of multi-dimensional arrays can be defined as follows:

```cpp
#pragma acc data create( A[0:N][0:M] )
```

If you do not include a starting index, then 0 is assumed. For example:

```cpp
#pragma acc data create( A[0:N] )
```

is equivalent to

```cpp
#pragma acc data create( A[:N] )
```

### Host or Device Memory?

Here are two loops:

```cpp
int *A = (int*) malloc(N * sizeof(int));

for (int i = 0; i < N; i++ )
{
    A[i] = 0;
}

#pragma acc parallel loop
for( int i = 0; i < N; i++ )
{
    A[i] = 1;
}
```

The first loop is not contained within an OpenACC compute region (a compute region is marked by either the parallel or kernels directive). Thus, `A[i]` will access host (CPU) memory.

The second loop is preceeded by the *parallel directive*, meaning that it is contained within an OpenACC compute region. `A[i]` in the second loop will access device (GPU) memory.

### Adding the Structured Data Directive to our Code

Use the following links to edit our laplace code. Add a structured data directive to properly handle the arrays `A` and `Anew`. 

[jacobi.c](jacobi.c)   
[laplace2d.c](laplace2d.c)  

Then, run the following script to check your solution. You code should run just as good as (or slightly better) than our managed memory code.

In [8]:
!nvc -fast -ta=tesla -Minfo=accel -o laplace_structured jacobi.c laplace2d.c && ./laplace_structured

jacobi.c:
main:
     59, Generating create(Anew[:m*n]) [if not already present]
         Generating copyin(A[:m*n]) [if not already present]
laplace2d.c:
calcNext:
     36, Generating copy(A[:n*m]) [if not already present]
         Generating Tesla code
         40, #pragma acc loop gang /* blockIdx.x */
             Generating implicit reduction(max:error)
         43, #pragma acc loop vector(128) /* threadIdx.x */
     36, Generating implicit copy(error) [if not already present]
         Generating copy(Anew[:n*m]) [if not already present]
     43, Loop is parallelizable
swap:
     54, Generating copy(A[:n*m],Anew[:n*m]) [if not already present]
         Generating Tesla code
         57, #pragma acc loop gang /* blockIdx.x */
         60, #pragma acc loop vector(128) /* threadIdx.x */
     60, Loop is parallelizable
Jacobi relaxation Calculation: 4096 x 4096 mesh
    0, 0.250000
  100, 0.002397
  200, 0.001204
  300, 0.000804
  400, 0.000603
  500, 0.000483
  600, 0.000403
  700, 0.

If you are feeling stuck, or would like to check your answer, you can view the correct answer with the following link.

[jacobi.c](solutions/advanced_data/structured/jacobi.c)

### Optional: Profile the Code

In [9]:
!nsys profile -t openacc --stats=true --force-overwrite true -o laplace_structured ./laplace_structured

Jacobi relaxation Calculation: 4096 x 4096 mesh
    0, 0.250000
  100, 0.002397
  200, 0.001204
  300, 0.000804
  400, 0.000603
  500, 0.000483
  600, 0.000403
  700, 0.000345
  800, 0.000302
  900, 0.000269
 total: 1.506861 s
Processing events...
Saving temporary "/tmp/nsys-report-a329-176b-b327-ccb3.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-a329-176b-b327-ccb3.qdrep"
Exporting 60205 events: [=================================================100%]

Exported successfully to
/tmp/nsys-report-a329-176b-b327-ccb3.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls   Average    Minimum   Maximum           Name        
 -------  ---------------  ---------  ----------  --------  --------  --------------------
    91.8       1045022147       4001    261190.2         0    641280  cuStreamSynchronize 
     2.8         31496094          1  31496094.0

Let's check out the profiler's report. Once the profiling run has completed, download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](laplace_structured.qdrep) (choose *save link as*), and open it via the GUI. To view the profiler report locally, please see the section on [How to view the report](../../../module2/English/C/README.ipynb#viewreport).

Take a moment to explore the profiler, and when you're ready, let's zoom in on the very beginning of our profile.

![structured.PNG](../images/structured.png)

We can see that we have uninterupted computation, and all of our data movement happens at the beginning of the program. This is ideal, because we are avoiding data transers in the middle of our computation.

---

## OpenACC Unstructured Data Directives

There are two unstructured data directives:

**enter data**: Handles device memory allocation, and copies from the Host to the Device. The two clauses that you may use with `enter data` are `create` for device memory allocation, and `copyin` for allocation, and memory copy.

**exit data**: Handles device memory deallocation, and copies from the Device to the Host. The two clauses that you may use with `exit data` are `delete` for device memory deallocation, and `copyout` for deallocation, and memory copy.

The unstructured data directives do not mark a "data region", because you are able to have multiple `enter data` and `exit data` directives in your code. It is better to think of them purely as memory allocation and deallocation.

The largest advantage of using unstructured data directives is their ability to branch across multiple functions. You may allocate your data in one function, and deallocate it in another. We can look at a simple example of that:

```cpp
int* allocate(int size)
{
    int *ptr = (int*) malloc(size * sizeof(int));
    #pragma acc enter data create(ptr[0:size])
    return ptr;
}

void deallocate(int *ptr)
{
    #pragma acc exit data delete(ptr)
    free(ptr);
}

int main()
{
    int *ptr = allocate(100);
    
    #pragma acc parallel loop
    for( int i = 0; i < 100; i++ )
    {
        ptr[i] = 0;
    }
    
    deallocate(ptr);
}
```

Just like in the above code sample, you must first allocate the CPU copy of the array **before** you can allocate the GPU copy. Also, you must deallocate the GPU of the array **before** you deallocate the CPU copy.

### Adding Unstructured Data Directives to our Code

We are going to edit our code to use unstructured data directives to handle memory management. First, run the following script to reset your code to how it was before adding the structured data directive.

In [10]:
!cp ./solutions/basic_data/jacobi.c ./jacobi.c && cp ./solutions/basic_data/laplace2d.c ./laplace2d.c && echo "Reset Finished"

Reset Finished


Now edit the code to use unstructured data directives. To fully utilize the unstructured data directives, try to get the code working by only altering the **laplace2d.c** code.

[jacobi.c](jacobi.c)   
[laplace2d.c](laplace2d.c)  

Run the following script to check your solution. Your code should run as fast as our structured implementation.

In [11]:
!nvc -fast -ta=tesla -Minfo=accel -o laplace_unstructured jacobi.c laplace2d.c && ./laplace_unstructured

jacobi.c:
laplace2d.c:
initialize:
     33, Generating enter data copyin(Anew[:m*n],A[:m*n])
calcNext:
     37, Generating copy(A[:n*m]) [if not already present]
         Generating Tesla code
         39, #pragma acc loop gang /* blockIdx.x */
             Generating implicit reduction(max:error)
         42, #pragma acc loop vector(128) /* threadIdx.x */
     37, Generating implicit copy(error) [if not already present]
         Generating copy(Anew[:n*m]) [if not already present]
     42, Loop is parallelizable
swap:
     53, Generating copy(A[:n*m],Anew[:n*m]) [if not already present]
         Generating Tesla code
         55, #pragma acc loop gang /* blockIdx.x */
         58, #pragma acc loop vector(128) /* threadIdx.x */
     58, Loop is parallelizable
deallocate:
     68, Generating exit data delete(Anew,A)
Jacobi relaxation Calculation: 4096 x 4096 mesh
    0, 0.250000
  100, 0.002397
  200, 0.001204
  300, 0.000804
  400, 0.000603
  500, 0.000483
  600, 0.000403
  700, 0.0003

If you are feeling stuck, or would like to check your answer, you can view the correct answer with the following link.

[laplace2d.c](solutions/advanced_data/unstructured/laplace2d.c)

### Optional: Profile the Code

In [15]:
!nsys profile -t openacc --stats=true --force-overwrite true -o laplace_unstructured ./laplace_unstructured

Jacobi relaxation Calculation: 4096 x 4096 mesh
    0, 0.250000
  100, 0.002397
  200, 0.001204
  300, 0.000804
  400, 0.000603
  500, 0.000483
  600, 0.000403
  700, 0.000345
  800, 0.000302
  900, 0.000269
 total: 1.163867 s
Processing events...
Saving temporary "/tmp/nsys-report-55c3-eec6-581c-0b3d.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-55c3-eec6-581c-0b3d.qdrep"
Exporting 60244 events: [=================================================100%]

Exported successfully to
/tmp/nsys-report-55c3-eec6-581c-0b3d.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls   Average    Minimum   Maximum           Name        
 -------  ---------------  ---------  ----------  --------  --------  --------------------
    92.0       1044215885       4001    260988.7         0   1502823  cuStreamSynchronize 
     2.7         30831900          1  30831900.0

Let's check out the profiler's report. Once the profiling run has completed, download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](laplace_unstructured.qdrep) (choose *save link as*), and open it via the GUI. To view the profiler report locally, please see the section on [How to view the report](../../../module2/English/C/README.ipynb#viewreport).

Take a moment to explore the profiler, and when you're ready, let's zoom in on the very beginning of our profile.

![unstructured.PNG](../images/unstructured.png)

We can see that we have uninterupted computation, and all of our data movement happens at the beginning of the program. This is ideal, because we are avoiding data transers in the middle of our computation. If you also profiled the structured version of the code, you will notice that the profiles are nearly identical. This isn't surprising, since the structured and unstructured approach work very similarly at the hardware level. However, structured data regions may be easier in simple codes, whereas some codes might flow better when using an unstructured approach. It is up to the programmer to determine via analysis and profiling, which to use.

---

## OpenACC Update Directive

When we use the data directives, there exist two places where the programmer can transfer data between the host and the device. For the structured data directive we have the opportunity to transfer data at the beginning and at the end of the region. For the unstructured data directives, we can transfer data when we use the enter data and exit data directives.

However, there may be times in your program where you need to transfer data in the middle of a data region, or between an enter data and an exit data. In order to transfer data at those times, we can use the `update` directive. The update directive will explicitly transfer data between the host and the device. The `update` directive has two clauses:

**self**: The self clause will transfer data from the device to the host (GPU to CPU)  
**device**: The device clause will transfer data from the host to the device (CPU to GPU)

The syntax would look like:

`#pragma acc update self(A[0:N])`

`#pragma acc update device(A[0:N])`

All of the array shaping rules apply.

As an example, let's create a version of our laplace code where we want to print the array **A** after every 100 iterations of our loop. The code will look like this:

```cpp
#pragma acc data copyin( A[:m*n],Anew[:m*n] )
{
    while ( error > tol && iter < iter_max )
    {
        error = calcNext(A, Anew, m, n);
        swap(A, Anew, m, n);
        
        if(iter % 100 == 0)
        {
            printf("%5d, %0.6f\n", iter, error);
            for( int i = 0; i < n; i++ )
            {
                for( int j = 0; j < m; j++ )
                {
                    printf("%0.2f ", A[i+j*m]);
                }
                printf("\n");
            }
        }
        
        iter++;

    }
}
```

Let's run this code (on a very small data set, so that we don't overload the console by printing thousands of numbers).

In [13]:
!nvc -fast -ta=tesla -Minfo=accel -o laplace_no_update ./update/jacobi.c ./update/laplace2d.c && ./laplace_no_update 10 10

./update/jacobi.c:
main:
     60, Generating copyin(Anew[:m*n],A[:m*n]) [if not already present]
./update/laplace2d.c:
calcNext:
     36, Generating copy(A[:n*m]) [if not already present]
         Generating Tesla code
         38, #pragma acc loop gang /* blockIdx.x */
             Generating implicit reduction(max:error)
         41, #pragma acc loop vector(128) /* threadIdx.x */
     36, Generating implicit copy(error) [if not already present]
         Generating copy(Anew[:n*m]) [if not already present]
     41, Loop is parallelizable
swap:
     52, Generating copy(A[:n*m],Anew[:n*m]) [if not already present]
         Generating Tesla code
         54, #pragma acc loop gang /* blockIdx.x */
         57, #pragma acc loop vector(128) /* threadIdx.x */
     57, Loop is parallelizable
Jacobi relaxation Calculation: 10 x 10 mesh
    0, 0.250000
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.

We can see that the array is not changing. This is because the host copy of `A` is not being **updated** between loop iterations. Let's add the update directive, and see how the output changes.

```cpp
#pragma acc data copyin( A[:m*n],Anew[:m*n] )
{
    while ( error > tol && iter < iter_max )
    {
        error = calcNext(A, Anew, m, n);
        swap(A, Anew, m, n);
        
        if(iter % 100 == 0)
        {
            printf("%5d, %0.6f\n", iter, error);
            
            #pragma acc update self(A[0:m*n])
            
            for( int i = 0; i < n; i++ )
            {
                for( int j = 0; j < m; j++ )
                {
                    printf("%0.2f ", A[i+j*m]);
                }
                printf("\n");
            }
        }
        
        iter++;

    }
}
```

In [14]:
!nvc -fast -ta=tesla -Minfo=accel -o laplace_update ./update/solution/jacobi.c ./update/solution/laplace2d.c && ./laplace_update 10 10

./update/solution/jacobi.c:
main:
     60, Generating copyin(A[:m*n],Anew[:m*n]) [if not already present]
     68, Generating update self(A[:m*n])
./update/solution/laplace2d.c:
calcNext:
     36, Generating copy(A[:n*m]) [if not already present]
         Generating Tesla code
         38, #pragma acc loop gang /* blockIdx.x */
             Generating implicit reduction(max:error)
         41, #pragma acc loop vector(128) /* threadIdx.x */
     36, Generating implicit copy(error) [if not already present]
         Generating copy(Anew[:n*m]) [if not already present]
     41, Loop is parallelizable
swap:
     52, Generating copy(A[:n*m],Anew[:n*m]) [if not already present]
         Generating Tesla code
         54, #pragma acc loop gang /* blockIdx.x */
         57, #pragma acc loop vector(128) /* threadIdx.x */
     57, Loop is parallelizable
Jacobi relaxation Calculation: 10 x 10 mesh
    0, 0.250000
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
0.00 0.25 0.25 0.25 0.25 0.25 0.25

---

## Conclusion

Relying on managed memory to handle data management can reduce the effort the programmer needs to parallelize their code, however, not all GPUs work with managed memory, and it is also lower performance than using explicit data management. OpenACC gives the programmer two main ways to handle data management, structured and unstructured data directives. By using these, the programmer is able to minimize the number of data transfers needed in their program.

---

## Bonus Task

If you would like some additional lessons on using OpenACC, there is an Introduction to OpenACC video series available from the OpenACC YouTube page. The fifth video in the series covers a lot of the content that was covered in this lab.  

[Introduction to Parallel Programming with OpenACC - Part 5](https://youtu.be/0zTX7-CPvV8)  

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommended you go to your browsers File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well.

You can also execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
rm -f openacc_files.zip
zip -r openacc_files.zip *

**After** executing the above zip command, you should be able to download and save the zip file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](openacc_files.zip)

# Licensing
This material is released by NVIDIA Corporation under the Creative Commons Attribution 4.0 International (CC BY 4.0).